# Main imports and code

In [1]:
# check which gpu we're using
!nvidia-smi

Fri Mar 04 15:27:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 456.71       Driver Version: 456.71       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 3080   WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   60C    P0    49W / 340W |   1275MiB / 10240MiB |     28%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  

In [2]:
!pip3 install simpletransformers
!pip3 install tensorboardx
!pip3 install numpy requests nlpaug
!pip3 install gpy gpyopt

You should consider upgrading via the 'C:\Users\Kerry\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Kerry\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Kerry\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Kerry\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from urllib import request
import pandas as pd
import logging
import torch
from collections import Counter

In [4]:
# prepare logger
logging.basicConfig(level=logging.INFO)

transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# check gpu
cuda_available = torch.cuda.is_available()

print('Cuda available? ',cuda_available)

device = torch.device("cuda" if cuda_available else "cpu")

Cuda available?  True


In [5]:
if cuda_available:
  import tensorflow as tf
  # Get the GPU device name.
  device_name = tf.test.gpu_device_name()
  # The device name should look like the following:
  if device_name == '/device:GPU:0':
      print('Found GPU at: {}'.format(device_name))
  else:
      raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


# Fetch Don't Patronize Me! data manager module

In [6]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


In [7]:
# helper function to save predictions to an output file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

In [8]:
from dont_patronize_me import DontPatronizeMe

In [9]:
dpm = DontPatronizeMe('.', './task_4_test.tsv')

In [10]:
dpm.load_task1()
dpm.load_task2(return_one_hot=True)
dpm.load_test()

Map of label to numerical label:
{'Unbalanced_power_relations': 0, 'Shallow_solution': 1, 'Presupposition': 2, 'Authority_voice': 3, 'Metaphors': 4, 'Compassion': 5, 'The_poorer_the_merrier': 6}


# Load paragraph IDs

In [11]:
trids = pd.read_csv('train_semeval_parids-labels.csv')
teids = pd.read_csv('dev_semeval_parids-labels.csv')

In [12]:
trids.head()

,par_id,label
0,4341,"[1, 0, 0, 1, 0, 0, 0]"
1,4136,"[0, 1, 0, 0, 0, 0, 0]"
2,10352,"[1, 0, 0, 0, 0, 1, 0]"
3,8279,"[0, 0, 0, 1, 0, 0, 0]"
4,1164,"[1, 0, 0, 1, 1, 1, 0]"


In [13]:
trids.par_id = trids.par_id.astype(str)
teids.par_id = teids.par_id.astype(str)

# Rebuild training set (Task 1)

In [14]:
rows = [] # will contain par_id, label and text
for idx in range(len(trids)):  
  parid = trids.par_id[idx]
  # select row from original dataset to retrieve `text` and binary label
  text = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0]
  label = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].label.values[0]
  orig_label = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].orig_label.values[0]
  country = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].country.values[0]
  keyword = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].keyword.values[0]
  rows.append({
      'par_id': parid,
      'text': text,
      'label': label,
      'orig_label': orig_label,
      'country': country,
      'keyword': keyword
  })
  

In [15]:
trdf1 = pd.DataFrame(rows)
trdf1

,par_id,text,label,orig_label,country,keyword
0,4341,"The scheme saw an estimated 150,000 children f...",1,4,gb,poor-families
1,4136,Durban 's homeless communities reconciliation ...,1,2,za,homeless
2,10352,The next immediate problem that cropped up was...,1,4,lk,poor-families
3,8279,Far more important than the implications for t...,1,2,nz,vulnerable
4,1164,To strengthen child-sensitive social protectio...,1,4,gh,poor-families
...,...,...,...,...,...,...
8370,8380,Rescue teams search for survivors on the rubbl...,0,0,gb,refugee
8371,8381,The launch of ' Happy Birthday ' took place la...,0,0,ke,hopeless
8372,8382,"The unrest has left at least 20,000 people dea...",0,0,pk,homeless
8373,8383,You have to see it from my perspective . I may...,0,0,pk,hopeless


# Rebuild internal test set (Task 1)

In [16]:
rows = [] # will contain par_id, label and text
for idx in range(len(teids)):  
  parid = teids.par_id[idx]
  # select row from original dataset
  text = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0]
  label = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].label.values[0]
  orig_label = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].orig_label.values[0]
  country = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].country.values[0]
  keyword = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].keyword.values[0]
  rows.append({
      'par_id': parid,
      'text': text,
      'label': label,
      'orig_label': orig_label,
      'country': country,
      'keyword': keyword
  })
  

In [17]:
len(rows)

2094

In [18]:
tedf1 = pd.DataFrame(rows)
tedf1

,par_id,text,label,orig_label,country,keyword
0,4046,We also know that they can benefit by receivin...,1,3,us,hopeless
1,1279,Pope Francis washed and kissed the feet of Mus...,1,4,ng,refugee
2,8330,Many refugees do n't want to be resettled anyw...,1,2,ng,refugee
3,4063,"""Budding chefs , like """" Fred """" , """" Winston ...",1,4,ie,in-need
4,4089,"""In a 90-degree view of his constituency , one...",1,3,pk,homeless
...,...,...,...,...,...,...
2089,10462,"The sad spectacle , which occurred on Saturday...",0,0,gh,homeless
2090,10463,""""""" The Pakistani police came to our house and...",0,0,pk,refugee
2091,10464,"""When Marie O'Donoghue went looking for a spec...",0,0,ie,disabled
2092,10465,"""Sri Lankan norms and culture inhibit women fr...",0,1,lk,women


# Rebuild test set

In [19]:
test_df = dpm.test_set_df
test_df

,par_id,art_id,keyword,country,text
0,t_0,@@7258997,vulnerable,us,"In the meantime , conservatives are working to..."
1,t_1,@@16397324,women,pk,In most poor households with no education chil...
2,t_2,@@16257812,migrant,ca,The real question is not whether immigration i...
3,t_3,@@3509652,migrant,gb,"In total , the country 's immigrant population..."
4,t_4,@@477506,vulnerable,ca,"Members of the church , which is part of Ken C..."
...,...,...,...,...,...
3827,t_3893,@@20319448,migrant,jm,In a letter dated Thursday to European Commiss...
3828,t_3894,@@9990672,poor-families,au,They discovered that poor families with health...
3829,t_3895,@@37984,migrant,ca,"She married at 19 , to Milan ( Emil ) Badovina..."
3830,t_3896,@@9691377,immigrant,us,The United Kingdom is n't going to devolve int...


# RoBERTa Baseline for Task 1

In [20]:
import nlpaug.augmenter.word as naw
import matplotlib.pyplot as plt
import numpy as np
from enum import Enum
import os, os.path
from os.path import exists
import glob
import sklearn
import GPyOpt
import copy
import re

class Upsample(Enum):
    BASELINE = 0
    DUPLICATION = 1
    BACK_TRANSLATION = 2
    SYNONYMS = 3

languages = ['fr', 'de', 'es', 'it', 'fi', 'nl', 'sla', 'zh']

# Helper functions

def back_translate(sentences, language):
    back_translation_aug = naw.BackTranslationAug(
    from_model_name=f'Helsinki-NLP/opus-mt-en-{language}',
    to_model_name=f'Helsinki-NLP/opus-mt-{language}-en',
    device=device)
    return back_translation_aug.augment(sentences)

def preprocess_fn(sentence):
    contractions = [("don't", "do n't"), ("I'd", "I 'd"), ("didn't", "did n't")]
    for old, new in contractions:
        sentence = sentence.replace(old, new)
    # Remove duplicate speech marks
    regex1 = '("+\s?"+)'
    while re.search(regex1, sentence):
        sentence = re.sub(r'("+\s?"+)', r'"', sentence)
    # Add space between punctuation
    regex2 = r"(?!n't)(\S)([.|,|;|:|?|'|\"])"
    while re.search(regex2, sentence):
        sentence = re.sub(regex2, r"\1 \2", sentence)
    return sentence

def words(sentence):
    return len(sentence.split())

def create_backtranslation_tsv(pcldf):
    for language in languages:
        file_name = f'{language}_pcl.tsv'
        if exists(file_name):
            continue
        print(f"Creating {file_name}")
        pcldf_lang = pcldf[pcldf["text"].apply(lambda x: words(x) < 300)]
        pcldf_lang = pcldf_lang.reset_index(drop=True)
        translated = back_translate(pcldf_lang['text'].tolist(), language)
        pcldf_lang['text'] = pd.Series(translated)
        pcldf_lang.to_csv(file_name, sep='\t')

def upsample_training_data(upsample_type, preprocess=True):
    pcldf = trdf1[trdf1.label==1]
    ncldf = trdf1[trdf1.label==0]
    npos = len(pcldf)
    ratio = len(ncldf) // len(pcldf)

    if upsample_type == Upsample.DUPLICATION:
        upsampled_pcldf = pd.concat([pcldf] * ratio, ignore_index=True)
        training_set = pd.concat([upsampled_pcldf,ncldf], ignore_index=True)
    elif upsample_type == Upsample.BACK_TRANSLATION:
        create_backtranslation_tsv(pcldf)
        language_dfs = []
        for language in languages:
            file_name = f'{language}_pcl.tsv'
            csv = pd.read_csv(file_name, sep='\t', index_col=[0])
            language_dfs.append(csv)
        upsampled_pcldf = pd.concat([pcldf, *language_dfs], ignore_index=True)
        training_set = pd.concat([upsampled_pcldf,ncldf], ignore_index=True)
    elif upsample_type == Upsample.SYNONYMS:
        synonym_dfs = []
        for file_name in glob.glob('synonym_pcl_*.tsv'):
            synonym_dfs.append(pd.read_csv(file_name, sep='\t', index_col=[0]))
        upsampled_pcldf = pd.concat([pcldf, *synonym_dfs], ignore_index=True)
        training_set = pd.concat([upsampled_pcldf,ncldf], ignore_index=True)
    else:
        training_set = pd.concat([pcldf,ncldf[:npos*2]], ignore_index=True)
    training_set.par_id = training_set.par_id.astype(str)
    if preprocess:
        training_set['text'] = training_set['text'].apply(preprocess_fn)
    return training_set

def safe_open_w(path):
    ''' Open "path" for writing, creating any parent directories as needed.
    '''
    os.makedirs(os.path.dirname(path), exist_ok=True)
    return open(path, 'a')

In [21]:
for upsample in Upsample:
    sample = upsample_training_data(upsample)
    positive_sample = sample[sample.label == 1]
    negative_sample = sample[sample.label == 0]
    print(upsample.name, len(sample), len(positive_sample), len(negative_sample))

BASELINE 2382 794 1588
DUPLICATION 14727 7146 7581
BACK_TRANSLATION 14719 7138 7581
SYNONYMS 14719 7138 7581


In [22]:
test_par_id = "2385"

print("No Preprocess")
for upsample in Upsample:
    if upsample == Upsample.BASELINE:
        continue
    df = upsample_training_data(upsample, preprocess=False)
    print(df[df.par_id == test_par_id].text.tolist()[1], upsample.name)

No Preprocess
""" Bruce has done a fantastic job in training youngsters , especially the youngsters that come from the poorest of the poor families , who would never have even had an opportunity to even buy a cricket bat . """ DUPLICATION
"" Bruce did a fantastic job in training young people, especially young people who come from the poorest of poor families, who would never even have had the opportunity to buy a cricket stick." BACK_TRANSLATION
" " " Sir david bruce has done a fantastic line in training fry, particularly the youngsters that come from the poorest of the poor families, who would never have yet have an opportunity to even buy a cricket chiropteran. " " " SYNONYMS


In [23]:
print("Preprocess")

for upsample in Upsample:
    if upsample == Upsample.BASELINE:
        continue
    df = upsample_training_data(upsample, preprocess=True)
    print(df[df.par_id == test_par_id].text.tolist()[1], upsample.name)

Preprocess
" Bruce has done a fantastic job in training youngsters , especially the youngsters that come from the poorest of the poor families , who would never have even had an opportunity to even buy a cricket bat . " DUPLICATION
" Bruce did a fantastic job in training young people , especially young people who come from the poorest of poor families , who would never even have had the opportunity to buy a cricket stick . " BACK_TRANSLATION
" Sir david bruce has done a fantastic line in training fry , particularly the youngsters that come from the poorest of the poor families , who would never have yet have an opportunity to even buy a cricket chiropteran . " SYNONYMS


In [24]:
from transformers import logging
logging.set_verbosity_error()
import warnings
warnings.filterwarnings("ignore")

def train_evaluate_model(model, output_file=None, upsample=Upsample.BASELINE, preprocess=True):
    training_set = upsample_training_data(upsample, preprocess)[['text', 'label']]
    eval_df = tedf1[['text', 'label']]
    if preprocess:
        eval_df['text'] = eval_df['text'].apply(preprocess_fn)
    _, training_details = model.train_model(training_set, eval_df=eval_df, f1=sklearn.metrics.f1_score)
    # run predictions
    preds_task1, model_output = model.predict(eval_df.text.tolist())
    task1_test_labels = eval_df.label.tolist()
    res = model.compute_metrics(preds_task1, model_output, task1_test_labels, f1=sklearn.metrics.f1_score)
    print(res[0])
    if output_file is not None:
        with safe_open_w(output_file) as f:
            f.write(str(res[0])+'\n')
    return training_details

def train_evaluate_simple_transformer(model_type, model_name, classification_args, output_file=None, upsample=Upsample.BASELINE, repetitions=1, preprocess=True):
    for i in range(repetitions):
        print(f"Training model {model_name}, Run {i+1}")
        model = ClassificationModel(model_type, 
                                    model_name, 
                                    args = classification_args, 
                                    num_labels=2, 
                                    use_cuda=cuda_available)
        training_details = train_evaluate_model(model, output_file, upsample, preprocess)
    return model, training_details

def train_evaluate_debertav3_base(classification_args, output_file=None, upsample=Upsample.BASELINE, repetitions=1, preprocess=True):
    return train_evaluate_simple_transformer('debertav2', 'microsoft/deberta-v3-base', classification_args, output_file, upsample, repetitions, preprocess)

def train_evaluate_debertav3_large(classification_args, output_file=None, upsample=Upsample.BASELINE, repetitions=1, preprocess=True):
    return train_evaluate_simple_transformer('debertav2', 'microsoft/deberta-v3-large', classification_args, output_file, upsample, repetitions, preprocess)

def train_evaluate_roberta_base(classification_args, output_file=None, upsample=Upsample.BASELINE, repetitions=1, preprocess=True):
    return train_evaluate_simple_transformer('roberta', 'roberta-base', classification_args, output_file, upsample, repetitions, preprocess)

def train_evaluate_roberta_large(classification_args, output_file=None, upsample=Upsample.BASELINE, repetitions=1, preprocess=True):
    return train_evaluate_simple_transformer('roberta', 'roberta-large', classification_args, output_file, upsample, repetitions, preprocess)

def train_evaluate_big_bird_base(classification_args, output_file=None, upsample=Upsample.BASELINE, repetitions=1, preprocess=True):
    return train_evaluate_simple_transformer('bigbird', 'google/bigbird-roberta-base', classification_args, output_file, upsample, repetitions, preprocess)

def train_evaluate_albertv2_base(classification_args, output_file=None, upsample=Upsample.BASELINE, repetitions=1, preprocess=True):
    return train_evaluate_simple_transformer('albert', 'albert-base-v2', classification_args, output_file, upsample, repetitions, preprocess)

def train_evaluate_albertv2_large(classification_args, output_file=None, upsample=Upsample.BASELINE, repetitions=1, preprocess=True):
    return train_evaluate_simple_transformer('albert', 'albert-large-v2', classification_args, output_file, upsample, repetitions, preprocess)

In [25]:
def hyperparamater_optimisation(params, model_type, model_name):
    lr = params.squeeze()
    print("Learning rate", lr)
    task1_model_args = ClassificationArgs(num_train_epochs=5, 
                                      no_save=True,
                                      no_cache=True,
                                      overwrite_output_dir=True,
                                      learning_rate=lr)
    task1_model = ClassificationModel(model_type,
                                      model_name,
                                      args = task1_model_args,
                                      num_labels=2,
                                      use_cuda=cuda_available)
    training_set = upsample_training_data(Upsample.BASELINE, preprocess=False)
    task1_model.train_model(training_set[['text', 'label']])
    preds_task1, model_output = task1_model.predict(tedf1.text.tolist())
    task1_test_labels = tedf1.label.tolist()
    res = task1_model.compute_metrics(preds_task1, model_output, task1_test_labels, f1=sklearn.metrics.f1_score)
    print(res[0])
    return res[0]["f1"]

### Baseline Experiments on Different Model Types

In [ ]:
model_args = ClassificationArgs(num_train_epochs=3, 
                                      no_save=True, 
                                      no_cache=True, 
                                      overwrite_output_dir=True,
                                      learning_rate=1e-5)

output_dir = "baseline_tests"

fns = [
    (train_evaluate_debertav3_base, "debertav3_base"), 
    (train_evaluate_debertav3_large, "debertav3_large"), 
    (train_evaluate_roberta_base, "roberta_base"), 
    (train_evaluate_roberta_large, "roberta_large"), 
    (train_evaluate_big_bird_base, "big_bird_base"),
    (train_evaluate_albertv2_base, "albertv2_base"),
    (train_evaluate_albertv2_large, "albertv2_large")
]

for f, file_name in fns:
    f(model_args, output_file = f"./{output_dir}/{file_name}", preprocess=False)

### Learning Rate Bayesian Optimization on Promising Models

In [ ]:
from functools import partial

bounds = [{'name': 'learning_rate', 'type': 'continuous', 'domain': (1e-6, 1e-4)}]
max_iter = 10

models = [
    ('debertav2', 'microsoft/deberta-v3-base'), 
    ('debertav2', 'microsoft/deberta-v3-large'), 
    ('roberta', 'roberta-base'), 
    ('roberta', 'roberta-large')
]

for model_type, model_name in models:
    hp_test = GPyOpt.methods.BayesianOptimization(f=partial(hyperparamater_optimisation, model_type=model_type, model_name=model_name), domain=bounds, maximize=True)
    hp_test.run_optimization(max_iter = max_iter)
    learning_rate = hp_test.x_opt
    print(f"Best hyperparaters for model {model_name}: lr = {learning_rate}")

## Model 1

In [ ]:
model_1_args = ClassificationArgs(num_train_epochs=5,
                                    no_save=True,
                                    no_cache=True,
                                    overwrite_output_dir=True,
                                    learning_rate=1.85636216e-5)
preprocess = False

In [ ]:
train_evaluate_debertav3_base(model_1_args, output_file = f"./model_1.txt", repetitions=5, preprocess=preprocess)

### Tests for Batch Size

In [ ]:
output_dir = "batch_size_tests"

# Model 1 has batchsize 8
batch_sizes = [16, 32, 64, 128]
for batch_size in batch_sizes:
    args = copy.copy(model_1_args)
    args.train_batch_size = batch_size
    train_evaluate_debertav3_base(args, output_file = f"./{output_dir}/{batch_size}.txt", repetitions=5, preprocess=preprocess)

### Tests for Max Seq Length

In [ ]:
output_dir = "max_seq_length_tests"

# Model 1 has max_seq_length 128
max_seq_lengths = [256, 384, 512]
for max_seq_length in max_seq_lengths:
    args = copy.copy(model_1_args)
    args.max_seq_length = max_seq_length
    train_evaluate_debertav3_base(args, output_file = f"./{output_dir}/{max_seq_length}.txt", repetitions=5, preprocess=preprocess)

### Tests for Epochs

In [ ]:
output_dir = "epochs_tests"

args = copy.copy(model_1_args)
args.num_train_epochs = 20
args.evaluate_during_training = True

_, details = train_evaluate_debertav3_base(args, output_file = f"./{output_dir}/{args.num_train_epochs}.txt", preprocess=preprocess)
print(details)

# Tests for Data Augmentation

In [ ]:
output_dir = "data_augmentation_tests"

for upsample in Upsample:
    train_evaluate_debertav3_base(model_1_args, output_file = f"./{output_dir}/{upsample.name.lower()}_preprocess.txt", upsample=upsample, repetitions=1, preprocess=True)
    train_evaluate_debertav3_base(model_1_args, output_file = f"./{output_dir}/{upsample.name.lower()}.txt", upsample=upsample, repetitions=1, preprocess=False)

## Model 2

In [26]:
model_2_args = ClassificationArgs(num_train_epochs=10,
                                    no_cache=True,
                                    overwrite_output_dir=True,
                                    train_batch_size=16,
                                    max_seq_length=256,
                                    learning_rate=1.85636216e-5,
                                    use_early_stopping=True,
                                    evaluate_during_training=True,
                                    early_stopping_delta=0.01,
                                    early_stopping_metric="f1",
                                    early_stopping_metric_minimize=False,
                                    early_stopping_patience=5,
                                    evaluate_during_training_steps=1000)
upsample = Upsample.BACK_TRANSLATION
preprocess = False

In [27]:
model_2, _ = train_evaluate_debertav3_base(model_2_args, output_file = f"./model_2.txt", upsample=upsample, preprocess=preprocess)
# Train 3 models and save them to best_models/model1, best_models/model2, best_models/model3

Training model microsoft/deberta-v3-base, Run 1


INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
Epochs 0/10. Running Loss:    0.0805: 100%|██████████| 920/920 [03:27<00:00,  4.42it/s]
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
Epoch 2 of 10:  10%|█         | 1/10 [03:50<34:32, 230.32s/it]INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.



Epochs 1/10. Running Loss:    0.0154: 100%|██████████| 920/920 [03:36<00:00,  4.26it/s]
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
Epoch 3 of 10:  20%|██        | 2/10 [07:43<30:57, 232.24s/it]INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.



  0%|          | 5/2094 [00:05<37:06,  1.07s/it]
INFO:simpletransformers.classification.classification_model: No improvement in f1
INF

{'mcc': 0.4798147402519046, 'tp': 100, 'tn': 1814, 'fp': 81, 'fn': 99, 'auroc': 0.9022460587899923, 'auprc': 0.5719538001423192, 'f1': 0.5263157894736843}


In [28]:
def predict_test_set(model, df, preprocess=True):
    # run predictions
    text = df.text.tolist()
    if preprocess:
        text = list(map(preprocess_fn, text))
    preds_task1, _ = model.predict(text)
    return preds_task1

def majority_vote(xs):
    return list(map(lambda x: Counter(x).most_common(1)[0][0], xs))

def metrics(pred, real):
    f1 = sklearn.metrics.f1_score(pred, real)
    precision = sklearn.metrics.precision_score(pred, real)
    recall = sklearn.metrics.recall_score(pred, real)
    return (f1, precision, recall)

In [31]:
preds_task1 = predict_test_set(model_2, tedf1, preprocess)
f1, precision, recall = metrics(preds_task1, tedf1.label.tolist())
print(f"Model 2 - F1: {f1}, Precision: {precision}, Recall: {recall}")

Model 2 - F1: 0.5263157894736843, Precision: 0.5025125628140703, Recall: 0.5524861878453039


In [38]:
model_dir = 'best_models'
models=["model1", "model2", "model3"]

In [36]:
all_preds = []
for model_name in models:
    model = ClassificationModel('debertav2', f'{model_dir}/{model_name}', num_labels=2,use_cuda=cuda_available)
    preprocess = False
    preds_task1 = predict_test_set(model, tedf1, preprocess)
    all_preds.append(preds_task1)
    f1, precision, recall = metrics(preds_task1, tedf1.label.tolist())
    print(f"{model_name} - F1: {f1}, Precision: {precision}, Recall: {recall}")

final_model_preds = majority_vote(list(zip(*all_preds)))
f1, precision, recall = metrics(final_model_preds, tedf1.label.tolist())
print(f"Final Model - F1: {f1}, Precision: {precision}, Recall: {recall}")

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
100%|██████████| 262/262 [00:25<00:00, 10.36it/s]


model1 - F1: 0.5995893223819301, Precision: 0.7336683417085427, Recall: 0.5069444444444444


INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
100%|██████████| 262/262 [00:09<00:00, 26.22it/s]


model2 - F1: 0.592734225621415, Precision: 0.7788944723618091, Recall: 0.4783950617283951


INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
100%|██████████| 262/262 [00:10<00:00, 26.19it/s]

model3 - F1: 0.5790754257907543, Precision: 0.5979899497487438, Recall: 0.5613207547169812
Final Model - F1: 0.620985010706638, Precision: 0.7286432160804021, Recall: 0.5410447761194029


In [42]:
def final_model_pred(df):
    all_preds = []
    for model_name in models:
        model = ClassificationModel('debertav2', f'{model_dir}/{model_name}', num_labels=2,use_cuda=cuda_available)
        preprocess = False
        preds_task1 = predict_test_set(model, df, preprocess)
        all_preds.append(preds_task1)
    return majority_vote(list(zip(*all_preds)))

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
 13%|█▎        | 60/479 [00:02<00:17, 23.81it/s]


KeyboardInterrupt: 

In [ ]:
final_model_preds_test = final_model_pred(test_df)

In [41]:
labels2file([[k] for k in final_model_preds_test], 'task1.txt')

# Analysis

## Prepare submission

In [ ]:
!cat task1.txt | head -n 10

In [ ]:
!zip submission.zip task1.txt

## Analysis

In [ ]:
def word_count(sentence):
    return len(sentence.split())

tedf = tedf1.copy()
tedf['prediction'] = final_model_pred(tedf1)
tedf['word_count'] = tedf['text'].apply(word_count) 

for i in range(5):
    filtered_tedf = tedf[tedf.orig_label == str(i)]
    correct_filter = filtered_tedf.prediction == 0 if i < 2 else filtered_tedf.prediction == 1
    correct_tedf = filtered_tedf[correct_filter]
    print(f"Accuracy for {i}: {len(correct_tedf)}/{len(filtered_tedf.index)} = {len(correct_tedf.index) /len(filtered_tedf.index)}")
print("\n")
 
# Analysis of sequence lengths
bins = [0, 50, 100, 150, 200, 250, 300]

for (lower, upper) in zip(bins, bins[1:]):
    filtered_tedf = tedf[(tedf.word_count < upper) & (tedf.word_count >= lower)]
    correct_tedf = filtered_tedf[tedf.label == tedf.prediction]
    f1 = sklearn.metrics.f1_score(filtered_tedf.prediction, filtered_tedf.label)
    if len(filtered_tedf) > 0:
        print(f"Accuracy for word counts {lower} to {upper}: {len(correct_tedf)}/{len(filtered_tedf)} = {len(correct_tedf)/len(filtered_tedf)}")
    print("F1 for word counts", lower, "to" , upper, "=", f1)
print("\n")
 
# # Analysis of country
for country in set(tedf.country.tolist()):
    filtered_tedf = tedf[tedf.country == country]
    correct_tedf = filtered_tedf[tedf.label == tedf.prediction]
    f1 = sklearn.metrics.f1_score(filtered_tedf.prediction, filtered_tedf.label)
    print(f"Accuracy for {country}: {len(correct_tedf)}/{len(filtered_tedf)} = {len(correct_tedf)/len(filtered_tedf)}")
    print(f"F1 for {country} = {f1}")
print("\n")
 
# # Analysis of keywords
for keyword in set(tedf.keyword.tolist()):
    filtered_tedf = tedf[tedf.keyword == keyword]
    correct_tedf = filtered_tedf[tedf.label == tedf.prediction]
    f1 = sklearn.metrics.f1_score(filtered_tedf.prediction, filtered_tedf.label)
    print(f"Accuracy for {keyword}: {len(correct_tedf)}/{len(filtered_tedf)} = {len(correct_tedf)/len(filtered_tedf)}")
    print(f"F1 for {keyword} = {f1}")

In [ ]:
for keyword in ['women', 'in-need']:
    keyword_tedf = tedf[tedf.keyword == keyword]
    for i in range(5):
        filtered_tedf = keyword_tedf[keyword_tedf.orig_label == str(i)]
        correct_filter = filtered_tedf.prediction == 0 if i < 2 else filtered_tedf.prediction == 1
        correct_tedf = filtered_tedf[correct_filter]
        if len(filtered_tedf.index) > 0:
            print(f"Accuracy for {keyword} {i}: {len(correct_tedf)}/{len(filtered_tedf.index)} = {len(correct_tedf.index) /len(filtered_tedf.index)}")
    print("\n")
